# FF Convergence

In [ ]:
import sys
sys.path.insert(0, '../../src/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import tensorflow as tf
import random
import pickle as pkl

from qiskit.quantum_info import Operator, Statevector, random_unitary
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm
from math import ceil

from quantum_tools import circuit_to_matrix, partial_trace
from kraus_channels import isomery_to_kraus, KrausMap
from quantum_channel import ConcatChannel, DilutedChannel, IdentityChannel, ChoiMapStatic
from utils import numberToBase, kron
from spectrum import complex_spacing_ratio, channel_spectrum, Spectrum, mean_spacing, coat_spectrum
from analysis import spectrum_to_angular, spectrum_to_radial, hist_ensamble
from qiskit.circuit.library import iSwapGate, XGate, YGate, HGate, CXGate, RGate, RZGate
from quantum_circuits import integrable_circuit, nonintegrable_circuit, haar_random
from synthetic_data import generate_map_data, generate_spam_data, generate_spam_benchmark
from optimization import ModelQuantumMap, ModelSPAM, Logger, model_saver
from loss_functions import ProbabilityMSE, ProbabilityRValue, channel_fidelity_loss
from spam import SPAM, InitialState, POVMwQR as POVM, CorruptionMatrix

#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=1)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)


## Circuits

In [2]:
def sample_traceout_csr(n, n_anc, reps, circuit_type, L):
    channel_list = []
    spectra_list = []
    csr_list = []


    for _ in tqdm(range(reps)):
        circuit = circuit_type(n+n_anc, L)
        U = Operator(circuit).data

        kraus_map = isomery_to_kraus(U, rank = 2**n_anc)
        channel_list.append(kraus_map)
        
        spectrum = channel_spectrum(kraus_map)
        spectra_list.append(spectrum)
        csr = complex_spacing_ratio(spectrum, log=False)
        csr_list.append(csr)

    return  channel_list, spectra_list, csr_list

In [3]:
def fit_spam(inputs, 
             targets,
             num_iter = 3000,
             verbose = False):
    d = targets.shape[1]
    spam_model = SPAM(init = InitialState(d),
                    povm = CorruptionMatrix(d),
                    )

    spam_opt = ModelSPAM(spam_model, tf.keras.optimizers.Adam(learning_rate=0.01))
        
    spam_opt.pretrain(100, verbose=False)

    spam_opt.train(inputs = inputs,
                    targets = targets,
                    num_iter = num_iter,
                    verbose = verbose,
                )
    
    return spam_model
    

def fit_model(inputs, 
              targets,
              channel, 
              spam_model,
              num_iter = 3000,
              verbose=False):
    d = targets.shape[1]
    model = ModelQuantumMap(channel = KrausMap(d = d, 
                                        rank = d**2,
                                        spam = spam_model,
                                        ),
                    loss_function = ProbabilityMSE(),
                    optimizer = tf.optimizers.Adam(learning_rate=0.01),
                    logger = Logger(loss_function_list = [ProbabilityRValue(), channel_fidelity_loss], sample_freq=100),
                )
    model.train(inputs = inputs, 
                targets = targets,
                inputs_val = [inputs, None],
                targets_val = [targets, [channel]],
                N=500,
                num_iter = num_iter,
                verbose = verbose,)
    
    return model

## Free Fermion

### Generate Benchmarks

In [4]:
n = 4
d = 2**n

[channel_FF_list, spectra_FF_list, csr_FF_list] =  pkl.load(open("data/FF_synthetic_benchmark.pkl", "rb"))

### Generate Synthetic Data and Fit

In [6]:
model_list = []

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

for channel in channel_FF_list:
    
    spam_target = generate_spam_benchmark(n=4, c1=0.95, c2=0.95, type="CM")
    inputs_spam, targets_spam = generate_spam_data(spam_target, shots=12000)
    
    spam_model = fit_spam(inputs_spam,
                        targets_spam,
                        num_iter = 3000,
                        verbose = False)
        

    inputs_map, targets_map = generate_map_data(channel, 
                                                spam_target = spam_target,
                                                N_map=5000-6**n, 
                                                shots = 12000)

    model = fit_model(inputs_map,
                    targets_map,
                    channel,
                    spam_model,
                    num_iter = 3000,
                    verbose = False
                    )

    model_list.append(model)

model_saver(model_list, "model_list_FF_noisy_CM.model")


[0.9982621312852501, -0.9919559407258679]
[0.9987375936058006, -0.9914119402463746]
[0.9985503248464884, -0.9907353427403264]
[0.9983694085611405, -0.9905779173073676]
[0.9984765570573038, -0.9904009441808769]
[0.9985748334355564, -0.9908344718742901]
[0.9987045634317373, -0.9930763577160264]
[0.9984768021404866, -0.9907860328058737]
[0.9985264523953767, -0.9913003339344654]
[0.998617267195537, -0.9923528951276096]


## Chaotic Haar Random

### Generate Benchmarks

In [7]:
n = 4
d = 2**n

[channel_FF_list, spectra_FF_list, csr_FF_list] =  pkl.load(open("data/chaotic_synthetic_benchmark.pkl", "rb"))

### Generate Synthetic Data and Fit

In [8]:
model_list = []

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

for channel in tqdm(channel_FF_list):
    
    spam_target = generate_spam_benchmark(n=4, c1=0.95, c2=0.95, type="CM")
    inputs_spam, targets_spam = generate_spam_data(spam_target, shots=12000)
    
    spam_model = fit_spam(inputs_spam,
                        targets_spam,
                        num_iter = 3000,
                        verbose = False)
        

    inputs_map, targets_map = generate_map_data(channel, 
                                                spam_target = spam_target,
                                                N_map=5000-6**n, 
                                                shots = 12000)

    model = fit_model(inputs_map,
                    targets_map,
                    channel,
                    spam_model,
                    num_iter = 3000,
                    verbose = False
                    )

    model_list.append(model)

model_saver(model_list, "model_list_chaotic_noisy_CM.model")


  0%|          | 0/10 [00:00<?, ?it/s]

[0.9967181468494726, -0.9905640283894421]
[0.9967301312402701, -0.990079985274731]
[0.9967328910180139, -0.9894055567053933]
[0.9967043309374316, -0.9895504087863298]
[0.9967388338632779, -0.9899128472374944]
[0.9967579112387046, -0.9893929443952535]
[0.9967750304003491, -0.9897060976276444]
[0.9967763435376749, -0.989736536429881]
[0.9967683393860595, -0.9897327718678064]
[0.996668075381639, -0.9905561819911752]
